In [10]:
#install and import required libraries.
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import numpy as np
from time import sleep
from random import randint

In [11]:
# information to be in english language
header={"Accept=language":"en-US,en;q=0.5"}

In [12]:
# extracting information of pages
pages=np.arange(1,1749,50)

In [13]:
# Create empty lists for the data
Sno = []
Movie_Name = []
Director_Name = []
Director1 = []
Director2 = []
Director3 = []
Duration = []
Year = []
Ratings = []
Metascore = []

In [14]:
#creating a for loop for required pages
for page in pages:
    #getting required url
    page=requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc")
    soup = BeautifulSoup(page.text, 'html.parser')
    # Find movie data on the current page
    movie_data = soup.find_all('div', attrs={'class': 'lister-item-content'})
    sleep(randint(2,8))
    # Iterate over each movie on the current page
    for movie in movie_data:
        # Extract serial number
        Sno.append(len(Sno) + 1)
        
        # Extract movie name
        Movie_Name.append(movie.h3.a.text)
        
        # Extract director names
        directors = movie.find_all('p')[2].find_all('a')
        director_names = [director.text for director in directors][:-4]
        Director_Name.append(director_names)
         # Assign values to Director1, Director2, and Director3
        director_count = len(director_names)
        Director1.append(director_names[0] if director_count > 0 else "-")
        Director2.append(director_names[1] if director_count > 1 else "-")
        Director3.append(director_names[2] if director_count > 2 else "-")

        # Extract duration
        Duration.append(movie.find('span', attrs={'class': 'runtime'}).text)
        
        # Extract year
        Year.append(movie.find('span', attrs={'class': 'lister-item-year'}).text.strip('()'))
        
        # Extract rating
        Ratings.append(movie.strong.text)
        
        # Extract metascore
        metascore = movie.find('span', attrs={'class': 'metascore'})
        Metascore.append(metascore.text if metascore else "-")

In [15]:
df = pd.DataFrame({
    'Sno': Sno,
    'Movie Name': Movie_Name,
    'Director1': Director1,
    'Director2': Director2,
    'Director3': Director3,
    'Duration': Duration,
    'Year': Year,
    'Ratings': Ratings,
    'Metascore': Metascore
})

In [16]:
df

,Sno,Movie Name,Director1,Director2,Director3,Duration,Year,Ratings,Metascore
0,1,The Dark Knight,Christopher Nolan,-,-,152 min,2008,9.0,84
1,2,The Lord of the Rings: The Return of the King,Peter Jackson,-,-,201 min,2003,9.0,94
2,3,Spider-Man: Across the Spider-Verse,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,140 min,2023,8.9,86
3,4,Inception,Christopher Nolan,-,-,148 min,2010,8.8,74
4,5,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,-,-,178 min,2001,8.8,92
...,...,...,...,...,...,...,...,...,...
1745,1746,Demon Slayer the Movie: Mugen Train,Haruo Sotozaki,-,-,117 min,2020,8.2,72
1746,1747,Gangs of Wasseypur,Anurag Kashyap,-,-,321 min,2012,8.2,89
1747,1748,Baahubali 2: The Conclusion,S.S. Rajamouli,-,-,167 min,2017,8.2,-
1748,1749,Yojimbo,Akira Kurosawa,-,-,110 min,1961,8.2,93


In [30]:
# saving file to csv
df.to_csv('IMBD_Data1.csv', index=False)

 process Table 2

In [31]:
import csv
import requests
from bs4 import BeautifulSoup

# Send a GET request to the IMDb URL
url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, "html.parser")

# Extract the data
data = []
movie_elements = soup.find_all("div", class_="lister-item-content")
for movie_element in movie_elements:
    movie_name = movie_element.find("h3").find("a").text
    stars_element = movie_element.find("p", class_="").find_all("a")
    stars = [star.text for star in stars_element]

    votes_element = movie_element.find("p", class_="sort-num_votes-visible").find("span", attrs={"name": "nv"})
    votes = votes_element.text if votes_element else None

    genre_element = movie_element.find("span", class_="genre")
    genre = [g.strip() for g in genre_element.text.strip().split(",")] if genre_element else []

    gross_collection_element = movie_element.find("p", class_="sort-num_votes-visible").find_next_sibling("span")
    gross_collection = gross_collection_element.text.strip() if gross_collection_element else None

    popularity_element = movie_element.find("span", class_="lister-item-rank")
    popularity = popularity_element.text.strip(".") if popularity_element else None

    certification_element = movie_element.find("span", class_="certificate")
    certification = certification_element.text.strip() if certification_element else None

    # Bifurcate stars into subfields star1, star2, star3, star4
    star1 = stars[0] if len(stars) >= 1 else None
    star2 = stars[1] if len(stars) >= 2 else None
    star3 = stars[2] if len(stars) >= 3 else None
    star4 = stars[3] if len(stars) >= 4 else None

    # Bifurcate genre into subfields genre1, genre2, genre3
    genre1 = genre[0] if len(genre) >= 1 else None
    genre2 = genre[1] if len(genre) >= 2 else None
    genre3 = genre[2] if len(genre) >= 3 else None

    data.append([movie_name, star1, star2, star3, star4, votes, genre1, genre2, genre3, gross_collection, popularity, certification])

# Store the data in the second CSV file
filename = "IMDB_Data_2.csv"
with open(filename, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Movie Name", "Star1", "Star2", "Star3", "Star4", "Votes", "Genre1", "Genre2", "Genre3", "Gross Collection", "Popularity", "Certification"])
    writer.writerows(data)

print("Data for the second CSV file stored in", filename)

Data for the second CSV file stored in IMDB_Data_2.csv


In [32]:
filename = "IMDB_Data_2.csv"
data = []

with open(filename, "r", newline="", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Skip the header row

    for row in reader:
        data.append(row)

# Print the data
for row in data:
    print(row)

['The Dark Knight', 'Christopher Nolan', 'Christian Bale', 'Heath Ledger', 'Aaron Eckhart', '2,741,127', 'Action', 'Crime', 'Drama', '', '', 'PG-13']
['The Lord of the Rings: The Return of the King', 'Peter Jackson', 'Elijah Wood', 'Viggo Mortensen', 'Ian McKellen', '1,899,013', 'Action', 'Adventure', 'Drama', '', '', 'PG-13']
['Spider-Man: Across the Spider-Verse', 'Joaquim Dos Santos', 'Kemp Powers', 'Justin K. Thompson', 'Shameik Moore', '185,491', 'Animation', 'Action', 'Adventure', '', '', 'PG']
['Inception', 'Christopher Nolan', 'Leonardo DiCaprio', 'Joseph Gordon-Levitt', 'Elliot Page', '2,432,436', 'Action', 'Adventure', 'Sci-Fi', '', '', 'PG-13']
['The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', 'Elijah Wood', 'Ian McKellen', 'Orlando Bloom', '1,927,425', 'Action', 'Adventure', 'Drama', '', '', 'PG-13']
['The Lord of the Rings: The Two Towers', 'Peter Jackson', 'Elijah Wood', 'Ian McKellen', 'Viggo Mortensen', '1,713,772', 'Action', 'Adventure', 'Drama', '

In [33]:
df2 = pd.DataFrame(data, columns=header)

In [34]:
df2

,Movie Name,Star1,Star2,Star3,Star4,Votes,Genre1,Genre2,Genre3,Gross Collection,Popularity,Certification
0,The Dark Knight,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,"2,741,127",Action,Crime,Drama,,,PG-13
1,The Lord of the Rings: The Return of the King,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,"1,899,013",Action,Adventure,Drama,,,PG-13
2,Spider-Man: Across the Spider-Verse,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,"185,491",Animation,Action,Adventure,,,PG
3,Inception,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,"2,432,436",Action,Adventure,Sci-Fi,,,PG-13
4,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,"1,927,425",Action,Adventure,Drama,,,PG-13
5,The Lord of the Rings: The Two Towers,Peter Jackson,Elijah Wood,Ian McKellen,Viggo Mortensen,"1,713,772",Action,Adventure,Drama,,,PG-13
6,The Matrix,Lana Wachowski,Lilly Wachowski,Keanu Reeves,Laurence Fishburne,"1,971,133",Action,Sci-Fi,,,,R
7,Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,"1,329,834",Action,Adventure,Fantasy,,,PG
8,Soorarai Pottru,Sudha Kongara,Suriya,Paresh Rawal,Aparna Balamurali,"120,033",Action,Drama,,,,TV-MA
9,Terminator 2: Judgment Day,James Cameron,Arnold Schwarzenegger,Linda Hamilton,Edward Furlong,"1,131,068",Action,Sci-Fi,,,,R


In [35]:
# saving file to csv
df.to_csv('IMBD_Data1.csv', index=False)
df2.to_csv("IMDB_Data_2.csv", index=False)

In [36]:
output_path = "C:\shiietal\Downloads simplilearn\sonali project scrapping"

In [37]:
output_path2 = "C:\shiietal\Downloads simplilearn\sonali project scrapping"

In [38]:
output_file = "C:\shiietal\Downloads simplilearn\sonali project scrapping" + "IMBD_Data1.csv"


In [39]:
output_file2 = "C:\shiietal\Downloads simplilearn\sonali project scrapping" + "IMDB_Data_2.csv"

In [40]:
df

,Sno,Movie Name,Director1,Director2,Director3,Duration,Year,Ratings,Metascore
0,1,The Dark Knight,Christopher Nolan,-,-,152 min,2008,9.0,84
1,2,The Lord of the Rings: The Return of the King,Peter Jackson,-,-,201 min,2003,9.0,94
2,3,Spider-Man: Across the Spider-Verse,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,140 min,2023,8.9,86
3,4,Inception,Christopher Nolan,-,-,148 min,2010,8.8,74
4,5,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,-,-,178 min,2001,8.8,92
...,...,...,...,...,...,...,...,...,...
1745,1746,Demon Slayer the Movie: Mugen Train,Haruo Sotozaki,-,-,117 min,2020,8.2,72
1746,1747,Gangs of Wasseypur,Anurag Kashyap,-,-,321 min,2012,8.2,89
1747,1748,Baahubali 2: The Conclusion,S.S. Rajamouli,-,-,167 min,2017,8.2,-
1748,1749,Yojimbo,Akira Kurosawa,-,-,110 min,1961,8.2,93


In [41]:
df2

,Movie Name,Star1,Star2,Star3,Star4,Votes,Genre1,Genre2,Genre3,Gross Collection,Popularity,Certification
0,The Dark Knight,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,"2,741,127",Action,Crime,Drama,,,PG-13
1,The Lord of the Rings: The Return of the King,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,"1,899,013",Action,Adventure,Drama,,,PG-13
2,Spider-Man: Across the Spider-Verse,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,"185,491",Animation,Action,Adventure,,,PG
3,Inception,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,"2,432,436",Action,Adventure,Sci-Fi,,,PG-13
4,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,"1,927,425",Action,Adventure,Drama,,,PG-13
5,The Lord of the Rings: The Two Towers,Peter Jackson,Elijah Wood,Ian McKellen,Viggo Mortensen,"1,713,772",Action,Adventure,Drama,,,PG-13
6,The Matrix,Lana Wachowski,Lilly Wachowski,Keanu Reeves,Laurence Fishburne,"1,971,133",Action,Sci-Fi,,,,R
7,Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,"1,329,834",Action,Adventure,Fantasy,,,PG
8,Soorarai Pottru,Sudha Kongara,Suriya,Paresh Rawal,Aparna Balamurali,"120,033",Action,Drama,,,,TV-MA
9,Terminator 2: Judgment Day,James Cameron,Arnold Schwarzenegger,Linda Hamilton,Edward Furlong,"1,131,068",Action,Sci-Fi,,,,R
